# 개요
- 앞선 스텝에서 선발된 모델의 하이퍼 파라미터를 조절하여 모델의 성능을 올리는 작업을 진행

In [1]:
# 경고창 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df_heart = pd.read_csv('../data/heart_2020_cleaned.csv')
df_heart

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319790,Yes,27.41,Yes,No,No,7.0,0.0,Yes,Male,60-64,Hispanic,Yes,No,Fair,6.0,Yes,No,No
319791,No,29.84,Yes,No,No,0.0,0.0,No,Male,35-39,Hispanic,No,Yes,Very good,5.0,Yes,No,No
319792,No,24.24,No,No,No,0.0,0.0,No,Female,45-49,Hispanic,No,Yes,Good,6.0,No,No,No
319793,No,32.81,No,No,No,0.0,0.0,No,Female,25-29,Hispanic,No,No,Good,12.0,No,No,No


# 데이터 전처리

In [4]:
# sleeptime 전처리 적용
df_heart['SleepTime'] = df_heart['SleepTime'].apply(lambda x: 4 if x <= 4 else (11 if x>=11 else x))

In [5]:
# BMI 이상치 제거 적용
import numpy as np

# 제 1 사분위수(Q1)와 제 3 사분위수(Q3) 계산
Q1 = np.percentile(df_heart['BMI'], 25)
Q3 = np.percentile(df_heart['BMI'], 75)

# IQR 계산
IQR = Q3 - Q1

# 이상치를 정의하는 임계값 설정
lower_threshold = Q1 - 1.5 * IQR
upper_threshold = Q3 + 1.5 * IQR
print("이상치 제거 전 low 갯수 : ",len(df_heart['BMI']))
# 이상치 제거
df_heart = df_heart.query(f'BMI >= {lower_threshold} & BMI <= {upper_threshold}')
print("이상치 제거 후 low 갯수 : ",len(df_heart['BMI']))

이상치 제거 전 low 갯수 :  319795
이상치 제거 후 low 갯수 :  309399


In [6]:
# Diabetic 그룹 통합 적용
df_heart['Diabetic'] = df_heart['Diabetic'].apply(lambda x:'Yes' if x=='No, borderline diabetes' else ('No' if x=='Yes (during pregnancy)' else x))
df_heart['Diabetic'].unique()

array(['Yes', 'No'], dtype=object)

In [7]:
#### 라벨 인코딩 진행
data_column_list = ['HeartDisease','Smoking','AlcoholDrinking','Stroke','DiffWalking','Sex','AgeCategory','Race','Diabetic','PhysicalActivity','GenHealth','Asthma','KidneyDisease','SkinCancer']
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [8]:
for i in data_column_list:
    df_heart[i] = label_encoder.fit_transform(df_heart[i])
df_heart

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,0,16.60,1,0,0,3.0,30.0,0,0,7,5,1,1,4,5.0,1,0,1
1,0,20.34,0,0,1,0.0,0.0,0,0,12,5,0,1,4,7.0,0,0,0
2,0,26.58,1,0,0,20.0,30.0,0,1,9,5,1,1,1,8.0,1,0,0
3,0,24.21,0,0,0,0.0,0.0,0,0,11,5,0,0,2,6.0,0,0,1
4,0,23.71,0,0,0,28.0,0.0,1,0,4,5,0,1,4,8.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319789,0,22.22,0,0,0,0.0,0.0,0,0,0,3,0,1,0,8.0,0,0,0
319790,1,27.41,1,0,0,7.0,0.0,1,1,8,3,1,0,1,6.0,1,0,0
319791,0,29.84,1,0,0,0.0,0.0,0,1,3,3,0,1,4,5.0,1,0,0
319792,0,24.24,0,0,0,0.0,0.0,0,0,5,3,0,1,2,6.0,0,0,0


# 모델 학습

In [9]:
# target과 feature 분리
target = df_heart['HeartDisease']
df_features = df_heart.copy()
features = df_features.drop(columns='HeartDisease')

In [10]:
# oversampling
from imblearn.over_sampling import SMOTE
overSampling = SMOTE(sampling_strategy=0.8)
feature_oversample, target_oversample =  overSampling.fit_resample(features,target)
feature_oversample.shape, target_oversample.shape

((509776, 17), (509776,))

In [11]:
# train set과 test set 분리
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(feature_oversample, target_oversample, test_size=0.3, random_state=42)
features_train.shape, features_test.shape, target_train.shape, target_test.shape

((356843, 17), (152933, 17), (356843,), (152933,))

In [12]:
from sklearn.ensemble import RandomForestClassifier

|파라미터|내용|기본값|
|--|--|--|
|n_estimators|의사결정 트리의 개수,일반적으로 큰 값으로 설정하는 것이 좋음||
|criterion|분할 기준을 지정하는 매개변수,gini, entropy, log_loss|gini|
|max_depth|각 의사결정 트리의 최대 깊이|None(최대깊이)|
|min_samples_split|노드를 분할하기 위한 최소 샘플 수,작은 값은 더 많은 분할을 유도 but 과적합 가능성|2|
|min_samples_leaf|리프 노드에 필요한 최소 샘플 수,이 값보다 적은 수의 샘플이 있는 리프 노드는 분할되지 않음|1|
|max_features|의사결정 트리가 분할에 사용할 최대 특성 수, "sqrt", "log2", 정수값 또는 실수값으로 지정|auto(특성의 제곱근)|
|n-jobs|학습 및 예측에 사용되는 병렬 작업의 수,int|None|
|bootstrap|중복을 허용하여 샘플을 랜덤 하게 선택하는 방식,bool|True|
|random_state|랜덤 시드를 지정|None|

초기 기존 randomForestClassifier 모델
```
              precision    recall  f1-score   support

           0       0.91      0.90      0.91     84882
           1       0.87      0.90      0.88     68051

    accuracy                           0.90    152933
   macro avg       0.89      0.90      0.89    152933
weighted avg       0.90      0.90      0.90    152933
```

# n_estimators 값 조정
- 기본 =  100

In [147]:
randomForestClassifier = RandomForestClassifier(n_estimators=16,random_state=42)

In [148]:
randomForestClassifier.fit(features_train,target_train)

RandomForestClassifier(n_estimators=16, random_state=42)

In [149]:
randomForestClassifier_test_prdict = randomForestClassifier.predict(features_test)

In [150]:
from sklearn.metrics import classification_report
print('randomForestClassifier 모델')
print(classification_report(target_test,randomForestClassifier_test_prdict))

randomForestClassifier 모델
              precision    recall  f1-score   support

           0       0.90      0.90      0.90     84882
           1       0.87      0.88      0.88     68051

    accuracy                           0.89    152933
   macro avg       0.89      0.89      0.89    152933
weighted avg       0.89      0.89      0.89    152933



|n_estimators|0 f1-score|1 f1-score|종합차이|비고|
|--|--|--|--|--|
|기본(100)|0.90|0.87|||
|20|0.90|0.88|+0.01|값 상승|
|25|0.90|0.88|+0.01|20과 차이 없음, 20이하 값 테스트|
|15|0.90|0.87||기본값과 동일, 최적값 15~20사이 존재|
|17|0.90|0.88|+0.01|값 상승, 최적값 16 or 17|
|16|0.90|0.88|+0.01|17과 동일, 16이 최적값|

- 최적값의 score가 기본값의 score보다 0.01 상승, n_estimators 최적값 = 16

# max_depth 값 조절
- max_depth는 기본값이 최대 깊이이므로 최소값부터 시작하여 점진적 테스트 진행 

In [183]:
randomForestClassifier = RandomForestClassifier(n_estimators=16,random_state=42,max_depth=100)

In [184]:
randomForestClassifier.fit(features_train,target_train)

RandomForestClassifier(n_estimators=16, random_state=42)

In [185]:
randomForestClassifier_test_prdict = randomForestClassifier.predict(features_test)

In [186]:
from sklearn.metrics import classification_report
print('randomForestClassifier 모델')
print(classification_report(target_test,randomForestClassifier_test_prdict))

randomForestClassifier 모델
              precision    recall  f1-score   support

           0       0.90      0.90      0.90     84882
           1       0.87      0.88      0.88     68051

    accuracy                           0.89    152933
   macro avg       0.89      0.89      0.89    152933
weighted avg       0.89      0.89      0.89    152933



|max_depth|0 f1-score|1 f1-score|종합차이|비고|
|--|--|--|--|--|
|기본(None, 최대깊이)|0.90|0.88|||
|1|0.73|0.44|상당차이|기본값보다 성능이 떨어짐, 값을 상당부분 올려서 retest|
|15|0.82|0.80|약간차이|아직 차이가 상당부분 남, 값을 조금 더 올려서 retest|
|25|0.89|0.86|-0.04|값이 좁혀졌으나 계속 하회, 값을 좀더 올려서 retest|
|30|0.90|0.88||기본값 도달, 상위값 테스트|
|35|0.90|0.88||30과 차이없음, 좀더 올려서 retest|
|100|0.90|0.88||차이 없음, 기본값이 최대 효율|

- 최적값의 score가 기본값의 score와 같으므로 max_depth 파라미터를 조절하여 최적성능 도출 불가

# min_samples_split 값 조절
- 기본값은 2
- 값을 조절하여 테스트 진행

In [191]:
randomForestClassifier = RandomForestClassifier(n_estimators=16,random_state=42,min_samples_split=4)

In [192]:
randomForestClassifier.fit(features_train,target_train)

RandomForestClassifier(min_samples_split=4, n_estimators=16, random_state=42)

In [193]:
randomForestClassifier_test_prdict = randomForestClassifier.predict(features_test)

In [194]:
from sklearn.metrics import classification_report
print('randomForestClassifier 모델')
print(classification_report(target_test,randomForestClassifier_test_prdict))

randomForestClassifier 모델
              precision    recall  f1-score   support

           0       0.91      0.89      0.90     84882
           1       0.86      0.89      0.87     68051

    accuracy                           0.89    152933
   macro avg       0.88      0.89      0.89    152933
weighted avg       0.89      0.89      0.89    152933



|min_samples_split|0 f1-score|1 f1-score|종합차이|비고|
|--|--|--|--|--|
|기본(2)|0.90|0.88|||
|3|0.90|0.88||값의 차이없음, 상위값 테스트|
|4|0.90|0.87|-0.01|성능이 하락하였으므로 기본값(2) or 3 이 최적값|


- 최적값의 score가 기본값의 score와 같으므로 min_samples_split 파라미터를 조절하여 최적성능 도출 불가

# min_samples_leaf 값 조절
- 기본값 1
- 값을 올려가면서 조절

In [195]:
randomForestClassifier = RandomForestClassifier(n_estimators=16,random_state=42,min_samples_leaf=2)

In [196]:
randomForestClassifier.fit(features_train,target_train)

RandomForestClassifier(min_samples_leaf=2, n_estimators=16, random_state=42)

In [197]:
randomForestClassifier_test_prdict = randomForestClassifier.predict(features_test)

In [198]:
from sklearn.metrics import classification_report
print('randomForestClassifier 모델')
print(classification_report(target_test,randomForestClassifier_test_prdict))

randomForestClassifier 모델
              precision    recall  f1-score   support

           0       0.90      0.88      0.89     84882
           1       0.85      0.88      0.87     68051

    accuracy                           0.88    152933
   macro avg       0.88      0.88      0.88    152933
weighted avg       0.88      0.88      0.88    152933



|min_samples_leaf|0 f1-score|1 f1-score|종합차이|비고|
|--|--|--|--|--|
|기본(1)|0.90|0.88|||
|2|0.89|0.87|-0.03|성능이 하락하였으므로 기본값이 최적값|

- 최적값의 score가 기본값의 score와 같으므로 min_samples_leaf 파라미터를 조절하여 최적성능 도출 불가

# max_features 값 조절
- 기본값 auto(특성의 제곱근)
- "sqrt", "log2", 정수값 또는 실수값으로 지정

In [239]:
randomForestClassifier = RandomForestClassifier(n_estimators=16,random_state=42,max_features=16)

In [240]:
randomForestClassifier.fit(features_train,target_train)

RandomForestClassifier(max_features=16, n_estimators=16, random_state=42)

In [241]:
randomForestClassifier_test_prdict = randomForestClassifier.predict(features_test)

In [242]:
from sklearn.metrics import classification_report
print('randomForestClassifier 모델')
print(classification_report(target_test,randomForestClassifier_test_prdict))

randomForestClassifier 모델
              precision    recall  f1-score   support

           0       0.90      0.90      0.90     84882
           1       0.87      0.88      0.88     68051

    accuracy                           0.89    152933
   macro avg       0.89      0.89      0.89    152933
weighted avg       0.89      0.89      0.89    152933



|max_features|0 f1-score|1 f1-score|종합차이|비고|
|--|--|--|--|--|
|기본(auto)|0.90|0.88|||
|sqrt|0.90|0.88||차이없음|
|log2|0.90|0.88||차이없음|
|1|0.90|0.87|-0.01|효율 하락,값을 올려서 테스트|
|2|0.90|0.87|-0.01|효율 하락,값을 올려서 테스트|
|5|0.90|0.88||차이없음, 값을 더 올려서 테스트|
|10|0.90|0.88||차이없음, 값을 더 올려서 테스트|
|20|0.90|0.87|-0.01|효율 하락, 최적값 10~20 사이|
|15|0.90|0.88||차이없음, 최적값 15~20사이|
|17|0.90|0.87|-0.01|효율 하락, 최적값 16 or 15|
|16|0.90|0.88||차이없음|

- 최적값의 score가 기본값의 score와 같으므로 max_features 파라미터를 조절하여 최적성능 도출 불가

# criterion 값 조절
- 기본값 gini
- 테스트값 entropy

In [247]:
randomForestClassifier = RandomForestClassifier(n_estimators=16,random_state=42,criterion='log_loss')

In [248]:
randomForestClassifier.fit(features_train,target_train)

RandomForestClassifier(criterion='log_loss', n_estimators=16, random_state=42)

In [249]:
randomForestClassifier_test_prdict = randomForestClassifier.predict(features_test)

In [250]:
from sklearn.metrics import classification_report
print('randomForestClassifier 모델')
print(classification_report(target_test,randomForestClassifier_test_prdict))

randomForestClassifier 모델
              precision    recall  f1-score   support

           0       0.90      0.90      0.90     84882
           1       0.87      0.88      0.88     68051

    accuracy                           0.89    152933
   macro avg       0.89      0.89      0.89    152933
weighted avg       0.89      0.89      0.89    152933



|criterion|0 f1-score|1 f1-score|종합차이|비고|
|--|--|--|--|--|
|기본(auto)|0.90|0.88|||
|entropy|0.90|0.88||차이없음|
|log_loss|0.90|0.88||차이없음|

- 최적값의 score가 기본값의 score와 같으므로 criterion 파라미터를 조절하여 최적성능 도출 불가

# 결론

- n_estimators의 값을 조절하여 효율 0.01 상승
- 더 많은 효율 상승을 위해서는 데이터 전처리의 방식을 바꾸거나 데이터를 추가하는 방법을 사용해야 한다고 판단